In [ ]:
import dlib
from skimage import io
from imageio import imread
from scipy.spatial import distance, KDTree
import json
import urllib
from bs4 import BeautifulSoup as bs
import re

hdr = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.11 (KHTML, like Gecko) Chrome/23.0.1271.64 Safari/537.11',
       'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8',
       'Accept-Charset': 'ISO-8859-1,utf-8;q=0.7,*;q=0.3',
       'Accept-Encoding': 'none',
       'Accept-Language': 'en-US,en;q=0.8',
       'Connection': 'keep-alive'}

In [ ]:
sp = dlib.shape_predictor('../models/shape_predictor_68_face_landmarks.dat')
facerec = dlib.face_recognition_model_v1('../models/dlib_face_recognition_resnet_model_v1.dat')
detector = dlib.get_frontal_face_detector()

class FaceDetecting:

    descriptor = None
    
    def __init__(self, person_name: str, person_image: str, photo_link: str, person_info=''):
        self.name = person_name
        self.image = person_image
        self.info = person_info
        self.descriptor = self.find_descriptor()
        self.photo_link = photo_link
    
    def find_descriptor(self):
        dets = detector(self.image, 1)
        
        for k, d in enumerate(dets):
            shape = sp(self.image, d)
        
        return facerec.compute_face_descriptor(self.image, shape)
            

def euclidian_distance(vector1, vector2):
    return distance.euclidean(vector1, vector2)
    

In [ ]:
actors_array = []
wrong_actors = []
txt = open('actors2.txt', 'a')

def get_actors_from_page(link: str, from_page: int):
    bsp = bs(urllib.request.urlopen(link + str(from_page)).read())
    actors = bsp.find("div", {"class": "lister-list"}).find_all("div", {"class": "lister-item mode-detail"})
    for actor in actors:
        try:
            print(actor.find('img')['src'].split('@.')[0] + '@.1_V1_SY1000_CR0.jpg')
            fd = FaceDetecting(person_name=actor.find('h3').find('a').text, person_image=imread(actor.find('img')['src'].split('@.')[0] + '@._V1_SY1000_CR0.jpg'), photo_link=actor.find('img')['src'].split('@.')[0] + '@._V1_SY1000_CR0.jpg')
            actors_array.append((fd.name, fd.photo_link, str(list(fd.descriptor))))
            txt.write(fd.name + '|' + fd.photo_link + '|' + str(list(fd.descriptor)))
        except:
            print('Wrong image format for :' + actor.find('h3').find('a').text)
            wrong_actors.append(actor.find('h3').find('a').text)

    return bsp.find("a", {"class": "lister-page-next next-page"})['href']
               
link_part = 'search/name/?gender=male,female&start=1&ref_=rlm'

for i in range(1):
    print(link_part)
    actors_array = []
    link_part = get_actors_from_page('https://www.imdb.com/', link_part)

In [ ]:
A = []

for el in actors_array['actors']:
    A.append(el['descriptor'])


In [ ]:
list(A[0])

In [ ]:
tree = KDTree(A)

In [ ]:
print(tree.query(actors_array[40].descriptor, k=10))

In [ ]:
print(actors_array[40].image)